In [3]:
using Pkg
Pkg.precompile()
ENV["GENIE_ENV"] = "dev"
using SearchLight
using SearchLightPostgreSQL
using BitemporalPostgres
using Test
SearchLight.Configuration.load() |> SearchLight.connect



ArgumentError: ArgumentError: Package BitemporalPostgres not found in current path:
- Run `import Pkg; Pkg.add("BitemporalPostgres")` to install the BitemporalPostgres package.


In [ ]:
using Pkg
Pkg.precompile()
ENV["GENIE_ENV"] = "dev"
using SearchLight
using SearchLightPostgreSQL
using BitemporalPostgres
using Test
using TimeZones
@testset "BitemporalPostgres unit tests" begin

  w=Workflow()
  w.tsw_validfrom=ZonedDateTime(2014, 5, 30,21,0,1,1, tz"UTC")
  SearchLight.Configuration.load() |> SearchLight.connect
  t=TestDummyComponent()
  tr=TestDummyComponentRevision(description="blue")
  ts=TestDummySubComponent(ref_super=t.id)
  tsr=TestDummySubComponentRevision(description="blue")

  create_entity!(w)
  create_component!(t,tr,w)
  create_subcomponent!(t,ts,tsr,w)
  @test !isnothing(w.ref_history)
  @test w.is_committed ==0
  @test w.ref_version == tr.ref_validfrom
  @test w.ref_version == tsr.ref_validfrom
  commit_workflow!(w)
  @test w.is_committed ==1

  w2=Workflow(ref_history=w.ref_history,tsw_validfrom=ZonedDateTime(2015, 5, 30,21,0,1,1, tz"UTC"))
  tr2=copy(tr)
  tr.description="yellow"
  update_entity!(w2)
  update_component!(tr,tr2,w2)
  @test w2.ref_version == tr2.ref_validfrom
  commit_workflow!(w2)
  @test w2.ref_version == tr.ref_invalidfrom
  @test w2.is_committed ==1

  @test !isnothing(SearchLight.query("""
    SELECT i.* FROM histories h JOIN versions v ON v.ref_history = h.id
    JOIN validityIntervals i ON i.ref_version = v.id 
    JOIN testdummyComponents p ON p.ref_history = h.id
    JOIN testdummycomponentrevisions r ON r.ref_component = p.id AND r.ref_valid @> v.id 
    WHERE h.id = 1 and p.id = 1 AND  i.tsrworld @> TIMESTAMPTZ '2015-05-30 20:00:01.001+00' AND i.tsrdb @>  NOW() 
""")
